In [1]:
!pip install -U qdrant-client

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
client = QdrantClient(":memory:")
collection_name = "demo_collection"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=4, distance=models.Distance.COSINE),
)

True

In [3]:
client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=1,
            vector=[0.05, 0.61, 0.76, 0.74],
            payload={
                "city": "Berlin", 
                "price": 1.99,
            },
        ),
        models.PointStruct(
            id=2,
            vector=[0.19, 0.81, 0.75, 0.11],
            payload={
                "city": ["Berlin", "London"], 
                "price": 1.99,
            },
        ),
        models.PointStruct(
            id=3,
            vector=[0.36, 0.55, 0.47, 0.94],
            payload={
                "city": ["Berlin", "Moscow"], 
                "price": [1.99, 2.99],
            },
        ),
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [7]:
client.search(
    collection_name=collection_name,
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="city",
                match=models.MatchValue(
                    value="London",
                ),
            )
        ]
    ),
    query_vector=[0.2, 0.1, 0.9, 0.7]
)

[ScoredPoint(id=2, version=0, score=0.6660352808733915, payload={'city': ['Berlin', 'London'], 'price': 1.99}, vector=None)]

In [12]:
client.search(
    collection_name=collection_name,
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="city",
                match=models.MatchAny(any=["Berlin"]),
            )
        ]
    ),
    query_vector=[0.2, 0.1, 0.9, 0.7], 
    limit=3
)

[ScoredPoint(id=1, version=0, score=0.8946329315113456, payload={'city': 'Berlin', 'price': 1.99}, vector=None),
 ScoredPoint(id=3, version=0, score=0.8387250995794813, payload={'city': ['Berlin', 'Moscow'], 'price': [1.99, 2.99]}, vector=None),
 ScoredPoint(id=2, version=0, score=0.6660352808733915, payload={'city': ['Berlin', 'London'], 'price': 1.99}, vector=None)]